In [61]:
#python resample.py $IMAGE_TO_RESAMPLE $REFERENCE_IMAGE

In [62]:
#!jupyter nbconvert bpress_2023-05-13.ipynb --to python

In [63]:
"""
~~~ Script description ~~~
adapted from troubshoot_bpress_slopes_6-24-22
adapted from ATM_bpress_within-between_1-4-23_rest.py on 1/31/23
- changed bpress to exclude overlap
changed path, added sys.argv for multiple jobs 5/11/23
- roi_activations_2023-05-11.npy is the same dictionary as
  mvpa_results_2023-05-11.npy with more windows and better name  
5/13
- Iterate across subs 
this script specifically looks at 28 subs for dmPFC and rTPJ
5/24
- create function that normalizes over space instead of time
6/06
- reran everything
--- copied from MVPA/final
"""

'\n~~~ Script description ~~~\nadapted from troubshoot_bpress_slopes_6-24-22\nadapted from ATM_bpress_within-between_1-4-23_rest.py on 1/31/23\n- changed bpress to exclude overlap\nchanged path, added sys.argv for multiple jobs 5/11/23\n- roi_activations_2023-05-11.npy is the same dictionary as\n  mvpa_results_2023-05-11.npy with more windows and better name  \n5/13\n- Iterate across subs \nthis script specifically looks at 28 subs for dmPFC and rTPJ\n5/24\n- create function that normalizes over space instead of time\n6/06\n- reran everything\n--- copied from MVPA/final\n'

In [64]:
import nibabel as nib

from nilearn.input_data import NiftiMasker , MultiNiftiMasker

import nilearn as nil
"""
Plotting!
https://nilearn.github.io/plotting/index.html

"""
from nilearn import plotting
import numpy as np 
import os
import os.path
import scipy.io
import nibabel as nib
from nilearn.input_data import NiftiMasker
from nilearn.masking import compute_epi_mask
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import PredefinedSplit
from copy import deepcopy

import warnings
import sys  
import random
# import logging

import deepdish as dd
import numpy as np

import brainiak.eventseg.event
import nibabel as nib
from nilearn.input_data import NiftiMasker

import scipy.io
from scipy import stats
from scipy.stats import norm, zscore, pearsonr
from scipy.signal import gaussian, convolve
from sklearn import decomposition
from sklearn.model_selection import LeaveOneOut, KFold

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
import seaborn as sns 

sns.set(style = 'white', context='talk', font_scale=1, rc={"lines.linewidth": 2})

if not sys.warnoptions:
    warnings.simplefilter("ignore")


random.seed(10)

from brainiak import image, io
from scipy.stats import stats
import nibabel as nib
import numpy as np
from matplotlib import pyplot as plt
from brainiak import image, io

import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.model_selection import LeavePGroupsOut

import pandas as pd
from nilearn.plotting import plot_glass_brain

# Import machine learning libraries
from nilearn.input_data import NiftiMasker
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold, f_classif, SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from scipy.stats import sem
from copy import deepcopy
from sklearn.metrics import roc_auc_score
import statistics

from nilearn import datasets, plotting
from nilearn.input_data import NiftiSpheresMasker

from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.image import concat_imgs, resample_img, mean_img, index_img
from nilearn import image
from nilearn import masking
from nilearn.plotting import view_img
from nilearn.plotting import plot_design_matrix
from nilearn.reporting import get_clusters_table
from nilearn import input_data

# Functions 

In [65]:
## Expand / Label TRs
"""
0 = SM
1 = SC
2 = OM
3 = OC
4 = Re
requires list of labels ouputed by psychopy (column 1 - MM_self_title.started, etc.)
returns label list (order is preserved) and TR labels
"""

def label_lists(label, num_tr):
    b = [[]]
    a = []
    for i in label:
        # substring label in psychopy output
        # if the first three characters == M_s, etc, then add correct indext to string
        if i[1:4] == "M_s":
            a.append("SM")
            b.append([0]*num_tr)
        elif i[1:4] == "C_s":
            a.append("SC")
            b.append([1]*num_tr)        
        elif i[1:4] == "M_o":
            a.append("OM")
            b.append([2]*num_tr)
        elif i[1:4] == "C_o":
            a.append("OC")
            b.append([3]*num_tr)     
        else:
            a.append("Re")
            b.append([4]*num_tr)     
    return a, b[1:]
 

In [66]:
def find_cond_index(sub_ses_labels):
    """
    For the array of ordered run names (i.e.'Re', 'SM',) find the two indexes per condition
    """ 
    lab_inx = []

    a = []
    b = []
    c = []
    d = []
    e = []

    for i in enumerate(sub_ses_labels):
        if i[1] == "SM":
            # append the index according to where it appeared in the array
            a.append(i[0])
        if i[1] == "SC":
            b.append(i[0])
        if i[1] == "OM":
            c.append(i[0])
        if i[1] == "OC":
            d.append(i[0])

    # Create a dictionary where each key contains the appropriate indexes
    lab_indic = {
        'SM' : a,
        'SC' : b,
        'OM' : c,
        'OC' : d,
        'RE' : [0,9]
    }
    return lab_indic 

In [7]:
def load_epi_data(sub, ses, task,run, space):
  # Load MRI file
    if space == "MNI":
        epi_in = os.path.join(data_dir, sub, ses, 'func', "%s_%s_task-%s_run-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz" % (sub, ses, task,run))
    elif space == "T1":
        epi_in = os.path.join(data_dir, sub, ses, 'func', "%s_%s_task-%s_run-%s_space-T1w_desc-preproc_bold.nii.gz" % (sub, ses, task,run))
    else:
        print("wrong load epi input. check this function")
    epi_data = nib.load(epi_in)
    print("Loading data from %s" % (epi_in))
    return epi_data

def load_roi_mask(ROI_name, space):
    if space == "MNI":
        maskdir = os.path.join(rois_dir)    
        print("expected shape: 78, 93,65")
    elif space == "T1":
        maskdir = os.path.join(rois_dir+ "/T1")
        print("expected shape: 56, 72,53")
    else:
        print("wrong mask input. check this function")
    # load the mask
    maskfile = os.path.join(maskdir, "%s.nii" % (ROI_name))
    mask = nib.load(maskfile)
    print("mask shape: ", mask.shape)
    print("Loaded %s mask" % (ROI_name))
    return mask


In [8]:
def intersect_mask(sub, num_runs,reg, ses="ses-01",task="Attn"):
    # This is based off of 'load_data' function in template
    # Loads all fMRI runs into a matrix #
    """
    reg = T1 or MNI registration?
    norm_type = by Space or by Time? 
    """
    yoz = []
    print("Begin intersecting, yeehaw")
    for run in range(1, num_runs + 1):
        # Load epi data 
        epi = load_epi_data(sub,ses,task,run,reg)
        # Load ROI data
        roi_samp = compute_epi_mask(epi)
     #   print(roi_samp)
        #nifti_masker = NiftiMasker(mask_img=roi_samp)
        #maskedData = nifti_masker.fit_transform(epi)
        yoz.append(roi_samp)
    #print(concatenated_data)
    epi_data = nil.masking.intersect_masks(yoz)
    print("all done wit da intersextion (lol)")

    return epi_data

In [9]:
# find indices of where each condition occured in 10 runs for each sub
def fnd_indices(sub,behav_p):
    behav = pd.read_csv(os.path.join(behav_p, '%s_behav_cleaned.csv') % (sub))
    # Define the column in behav to be used for creating labels # 
    label = behav.iloc[:,1]
    # Create an array of labels [1] AND the order in which runs occured [0]#
    sub_ses_labels = label_lists(label, 200)
    ## Find run sequence, extraction condition indexes from behav data ## 
    return find_cond_index(sub_ses_labels[0])

In [10]:
# organize into dict
def org_bdata_dic(unsort_bdata, run_indexes, cond_a, cond_b): 
    """
    organize two runs for concatenation
    Two runs of cond_a, then two runs of cond_b, in a dik
    
    """
    bold_dict = {}
    a = [unsort_bdata[run_indexes[cond_a][0]], unsort_bdata[run_indexes[cond_a][1]]]
    b = [unsort_bdata[run_indexes[cond_b][0]], unsort_bdata[run_indexes[cond_b][1]]]
    bold_dict[cond_a] = a
    bold_dict[cond_b] = b
    
    print("concatenated", cond_a, cond_b)
    return bold_dict

# organize into list
def org_bdata_list(unsort_bdata, run_indexes, cond_a, cond_b): 
    """
    organize two runs for concatenation
    Two runs of cond_a, then two runs of cond_b
    
    """
    bold_data = []
    a = [unsort_bdata[run_indexes[cond_a][0]], unsort_bdata[run_indexes[cond_a][1]]]
    b = [unsort_bdata[run_indexes[cond_b][0]], unsort_bdata[run_indexes[cond_b][1]]]
    print("concatenated", cond_a, cond_b)
    return a + b

def org_bdata(unsort_bdata, run_indexes, cond_a, cond_b): 
    """
    organize two runs for concatenation
    Two runs of cond_a, then two runs of cond_b
    
    """
    bold_data = []
    a = [unsort_bdata[run_indexes[cond_a][0]], unsort_bdata[run_indexes[cond_a][1]]]
    b = [unsort_bdata[run_indexes[cond_b][0]], unsort_bdata[run_indexes[cond_b][1]]]
    print("returning", cond_a, cond_b)
    return np.asarray(a), np.asarray(b)

In [11]:
suffix = "_205_noproc.npy"

# load data into a dictionary
def load_smsc_fmri_list(sub,behav_p,cond_a,cond_b,suffix,int_mask):
    cats = list(np.load(load_fmri + sub + suffix, allow_pickle =True))
    #cats = load_fMRI3d(sub, 10, "MNI", 'space', int_mask)
    # Find run labels from behavioral data
    lab_indic = fnd_indices(sub, behav_p)
    # Organize and concatenate bold data
    return org_bdata_list(cats, lab_indic, cond_a, cond_b)

# Load data into dictionary
def load_smsc_fmri_dic(sub,behav_p,cond_a,cond_b,suffix,int_mask):
    cats = list(np.load(load_fmri + sub + suffix, allow_pickle =True))
    #cats = load_fMRI3d(sub, 10, "MNI", 'space', int_mask)
    # Find run labels from behavioral data
    lab_indic = fnd_indices(sub, behav_p)
    # Organize and concatenate bold data
    return org_bdata_dic(cats, lab_indic, cond_a, cond_b)

# Rest stuff

In [12]:
def create_event_list_rest(sub, bpress, cond, cond_alt, run_dic, base_onset,comp_onset_list,stim_dur):
    """
    this function reads in a condition for each sub
    and returns the corresponding b4 + after events
    
    sub: subject number
    bpress: array of button press onset times
    cond: which condition do you want to create event dataframe fore
    """
    all_tims = []
    events = {}
    # Convert alternate bpress to array #
    bpress_arr = np.asarray(bpress[sub][cond_alt])
    # Select runs to include according to run_dic, append
    # * include runs of rest 
    all_tims = all_tims + list(bpress_arr[run_dic[sub]['RE']])
    # how much to shift from button press onset
    
    ## create fake bpress for missing data if only one run ## 
    if len(all_tims) <2 and len(run_dic[sub]['RE']) >1:
        all_tims = list(all_tims[0], all_tims[0])
        
    return all_tims

In [13]:
def load_2conds_runs_fmri(sub,behav_p,cond_a,cond_b, run_dic, suffix="_205_noproc.npy"):
    """
    read in target conditions + subject info
    output: 2 runs of condition A, as they were presented, then two runs of condition B
    """
    cats = list(np.load(load_fmri + sub + suffix, allow_pickle =True)) 
    # Find run labels from behavioral data
    lab_indic = fnd_indices(sub, behav_p)
    # Organize and concatenate bold data
    a, b = org_bdata(cats, lab_indic, cond_a, cond_b)
    # Get the indices of the runs to include #
    return list(a[run_dic[sub][cond_a]]), list(b[run_dic[sub][cond_b]])

In [14]:
def load_1cond_runs_fmri(sub,behav_p,cond_a, cond_b, run_dic, targ_run, suffix="_205_noproc.npy"):
    """
    read in target conditions + subject info
    output: 2 runs of condition A, as they were presented, then two runs of condition B
    """
    cats = list(np.load(load_fmri + sub + suffix, allow_pickle =True))
    # Find run labels from behavioral data
    lab_indic = fnd_indices(sub, behav_p)
    # Organize and concatenate bold data
    a, b = org_bdata(cats, lab_indic, cond_a, cond_b)
    # Get the indices of the runs to include #
    return list(a[run_dic[sub][cond_a]])

# Define Static VARS

In [67]:
top_dir = '/jukebox/graziano/coolCatIsaac/MEI'
data_dir = top_dir + "/data"
work_dir = data_dir + '/work'
mask_dir = work_dir + '/masks'
behav_dir = top_dir + '/data/behavioral'
rois_dir = data_dir + "/rois"
fmri_prep = data_dir + '/bids/derivatives/fmriprep'
conf_dir = work_dir + '/confs'
preproc_dir = work_dir + '/preproc'
isc_dir = work_dir + '/isc_dat'


# dynamic vars

In [68]:
### sub_list ###
"""

========================================================
===== ~~~ Summary ALL ~~~ ===== 
========================================================
Initially exclude: sub-001, sub-003, sub-011, sub-012, sub-014, sub-015, sub-029 (3+ runs still available tho)
Maybe exclude: sub-014, sub-024, sub-037 (No button presses)
Maybe exclude: sub-020, sub-022, sub-031
total exclude = 10


~~~~~~~ BEHAVIORAL ~~~~~~~~~~
~~~ Summary behavioral ~~~

========================================================
========================================================
No button presses: sub-014, sub-024, sub-037 

========================================================
========================================================
sub-003: Extra row observed for external, which was manually deleted. Should be fine to rerun -- i removed from bad_subs
    - current brushing: 87, TRUE: 88 for THREE runs. last run when there was an issue and scan had to be restarted
    - Scan 6 == 1 run of brushing
    - Scan 7 = 3 runs of brushing 
    - ** no idea when run 7 was started tho! will prolly need to throw out! For now we can process all
sub-014: NO internal OR external bpress, curious! ---should be fixed now
    - changed excel data to include two null columns
    - ALSO only 30 TRs for the second run epi data. i went into the room to adjust runny eyes. Data for this run is not usable. but after should
      be okay if u want to put in the effort 
sub-015: less than 24 runs ---should be fixed now w custom code (but keep out of main run cuz will throw an error otherwise )
    - SUMMARY: need to exclude the office run 3 for fMRI but can include run 3 in behavioral, no data at all for CAKE
    - DID NOT add the 'mov_name' component to the custom code, will need to implment from current iteration
    - **** Use external, ignore internal for now... cuz internal is fuqd. Can preprocess all 7 tho

~~~~~~~ FMRI ~~~~~~~~~~
~~~ Summary fMRI ~~~
sub002: 
 - appended two TRs onto the last run, copied from the third to last TR. should be good to use
sub-003, sub-012, sub-014, sub-015, sub-029: 
 - Five runs usable, will need to incorporate them at some point 
sub-001: external usable - first three runs 

========================================================
========================================================
subject 002: (usable, with adjustment)
    only has 92 INSTEAD OF 94 TRs for the FINAL run of shrek. i think cuz i turned off the scanner
    too soon, which didn't account for 4 TRs of buffer? Yep! end tr is 390, (388, 112179). so duplicate TRs maybe.
    - Temporarily eliminate! or duplicate TRs 
    
sub-003: (usable, 5 runs)
- Scan 6 == 1 run of brushing
- Scan 7 = 3 runs of brushing 
*** need to re-preprocess, then append 6 and 7 together; or just throw out this one cuz idk when scan started

sub-011: (usable)
- not preprocessed 
sub-012 (5 runs usable, one run idk)
- I'll need to post process. It seems that an earbud fell out while reading the directions for run 5 
summary: the fifth run  needs to be discarded. all others are usable.
see the behav data: 
sub_dic['External']['oragami']
{'run-1': {'start_tr': 94, 'end_tr': 188, 'bpress': -1},
 'run-2': {'start_tr': 188, 'end_tr': 282, 'bpress': [100.81657150003593]},
 'run-3': {'start_tr': 282, 'end_tr': 376, 'bpress': [88.26579949003644]},
 'run-4': {'start_tr': 378,
  'end_tr': 472,
  }
- you can see taht the start TR is 94!! that's becuz i had to go into the scan room during the instructions screen.
- this is producing the wrong indexing -- becuz the scan doesn't contain enough TRs according to the behavioral data:
    - run 4 of shrek only has 12 TRs because we start at TR 94.
- im not sure when the scan actually started, but if i wanted to try to include i could 
    set the start TR to 1 -- essentially suggesting that the first TR was collected on the external 'waiting 
    for TR' flip. might as well try at some point, but exclude for now
    - this would involve just subtracting 94 from all start end end TRs i think
See the excel behavioral file for further notes notes
this 

sub-014 - no button press data and 7 runs (5 runs usable, but no bpress)
    - same problem as above -- i went into the scanner room during the instructions period to wipe eyes
    {'run-1': {'start_tr': 237, 'end_tr': 323, 'bpress': -1},
     'run-2': {'start_tr': 329, 'end_tr': 415, 'bpress': -1},
     'run-3': {'start_tr': 417, 'end_tr': 503, 'bpress': -1},
     'run-4': {'start_tr': 509, 'end_tr': 595, 'bpress': -1}}

sub-029: (5 runs usable)
- scanner malfunction, maybe possible to stitch together run 6 cuz i started on the *tenth* TR

========================================================
========================================================
Left handed: (sub-020, sub-022, sub-032)
    - sub-020: also no button presses, appeared to be awake tho
    - sub-022: lefty
    - sub-031: ambidexterious, great data
========================================================
========================================================
"""

## adjusted sublist 
sub_list = [
    'sub-002', 'sub-004', 'sub-005','sub-006','sub-007','sub-008','sub-009','sub-010',
    'sub-013','sub-016','sub-017','sub-018','sub-019','sub-020','sub-021',
    'sub-022','sub-023','sub-024','sub-025','sub-026','sub-027','sub-028','sub-030','sub-031','sub-032',
    'sub-033','sub-034','sub-035','sub-036','sub-037','sub-038','sub-039','sub-040'
]
#sub_list = ['sub-002', 'sub-003', 'sub-004', 'sub-005']

###### LOADING VARS #######




In [69]:
def nada(bpress_arr):
    """
    purpose: return 0 if no bpresses
    input: take in an array of bpresses
    output: returns the length of the bpress array -- number of button presses
    """
    if bpress_arr == -1: return 0
    else: return len(bpress_arr)

In [37]:
## BEHAVIORAL ##
sub_dic_behav = np.load(os.path.join(behav_dir, f'sub-005_behav.npy'), allow_pickle=True).item()

In [39]:
x = sub_dic_behav['Internal']['sherlock']['run-1']['bpress']

In [44]:
x

[53.58145424100803, 70.86647686501965, 100.8213532959926, 135.343493072025]

In [45]:
np.diff(x)

array([17.28502262, 29.95487643, 34.52213978])

In [34]:
counts = []
mov_dic = {}
for sub in sub_list:
    sub_dic_behav = np.load(os.path.join(behav_dir, f'{sub}_behav.npy'), allow_pickle=True).item()
    for mov in sub_dic_behav['Internal']:
        bpress_arr = sub_dic_behav['Internal'][mov]['run-1']['bpress']
        if mov not in mov_dic:
            mov_dic[mov] = [nada(bpress_arr)]
        else:
            mov_dic[mov].append(nada(bpress_arr))
        
        
        

In [28]:
sub_dic_behav['Internal'][mov]['run-1']['bpress']

-1

In [48]:
sub_dic_fmri = np.load(f'{preproc_dir}/{sub}_fwhm6_conf.npy', allow_pickle=True).item()

In [50]:
sub_dic_fmri[1].shape

(450, 112179)

In [58]:
m1 = np.array([[1,20], [30,40]])

In [59]:
m2 = np.array([[7,14], [22,44]])

In [60]:
np.mean([m1, m2], axis =0)

array([[ 4., 17.],
       [26., 42.]])

In [47]:
tr_range = np.arange(-6,13)
tr_range

array([-6, -5, -4, -3, -2, -1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
       11, 12])

In [ ]:
## load fMRI
sub_dic_fmri = np.load(f'{preproc_dir}/{sub}_fwhm6_conf.npy', allow_pickle=True).item()

## load behavioral
sub_dic_behav = np.load(os.path.join(behav_dir, f'{sub}_behav.npy'), allow_pickle=True).item()

## if no button presses for this run, skip!
if bpress_arr == -1: continue




# Loop over movie repetitions (there are 4)
for run in range(1, 5):
    # Is this an internal or external run?
    key = 'External' if (sub_num % 2 == 1 and epi_index < 3) or (sub_num % 2 == 0 and epi_index >= 3) else 'Internal'

    ## get behavioral data
    bpress_arr = sub_dic_behav['Internal'][mov][run]['bpress']
    
    # calculate differences
    difs = np.difs(bpress_arr)
    
    # Begin slicing fMRI data #
    start_tr = sub_dic_behav[key][mov_name][f'run-{run:d}']['start_tr']
    end_tr = sub_dic_behav[key][mov_name][f'run-{run:d}']['end_tr']
    run_slice = fmri_run[start_tr:end_tr, :]
    print(f'fMRI shape: {run_slice.shape}')
    
    ## empty array to append if TR doesn't exist or conflicts with another button pres!
    empty = np.zeros((run_slice.shape[1])) ## number of voxels

    
    # For each button press
    # create button press matrix. should be bpress x 12 for each button press
    bpress_mat = []
    for bpress in bpres_arr:
        # Find the tr that each onset occured - convert from seconds to TR
        print(f'time of bpress {bpress}')
        bpress_tr = round(bpress/1.5)
        assert bpress_tr < end_tr, print('TR extends past fMRI data, what on earth is going on!')
        print(bpress_tr)
        temp = []
        ## for each tr in the TR range [likely this is 12]
        for num_tr, idx in enumerate(tr_range):
            try:
                # IF tr exists in the range of TRs add to temp array before averaging in the next step
                current_bpress_tr = run_slice[bpress_tr+num_tr]
                # if no error is thrown, run a check
                # if the distance between the current and next tR is less than 1.5*6 seconds, do not extract that data
                # and fill with empty zeros to be averaged over later
                # 6 is the number of TRs ahead. we convert to seconds by multiplying by 1.5
                if difs[idx] < 1.5 * 6:
                    temp.append(empty)
                else:
                    # add the correct voxel activation for the given TR 
                    temp.append(current_bpress_tr)
            except:
                # if it doesn't exist, add an empty vector
                temp.append(empty)
        bpress_mat.append(temp)
    ## once we've finished iterating over all button press, average each bpress matrix element-wise!
    bpress_mat_av = np.mean(bpress_mat, axis =0)
    # convert back to nifti and save 
        
                
        
    
    
    

In [ ]:
### nothing is below

In [ ]:
external = {}
internal = {}

for sub in sub_list:
    ### fMRI load ###
    sub_dic_fmri = np.load(f'{preproc_dir}/{sub}_fwhm6_conf_4D.npy', allow_pickle=True).item()
    print(f'start {sub}')
    ## BEHAVIORAL ##
    sub_dic_behav = np.load(os.path.join(behav_dir, f'{sub}_behav.npy'), allow_pickle=True).item()
    
    # Create subject number 
    sub_num = int(sub[-3:])

    for epi_index in range(0, 6):
        # Add one to the index to create 1-6 runs
        epi_run = epi_index + 1

        # Get the movie name
        mov_name = sub_dic_behav['mov_order'][epi_index]

        # Create an empty array for the movie runs, append four TRs to account for the 4 trailing TRs, subtract
        # the quantity of TRs that we are trimming from the front 
        mov_runs = np.zeros((mov_len_dic[mov_name] + tr_buffer - trim_start, num_parc, 0))

        print(f'movie: {mov_name} with shape {mov_runs.shape}')

        # Get the fMRI run for the current epi_index
        fmri_run = sub_dic_fmri[epi_run]
        
        # Loop over movie repetitions (there are 4)
        for run in range(1, 5):
            # Is this an internal or external run?
            key = 'External' if (sub_num % 2 == 1 and epi_index < 3) or (sub_num % 2 == 0 and epi_index >= 3) else 'Internal'
            
            # Begin slicing #
            start_tr = sub_dic_behav[key][mov_name][f'run-{run:d}']['start_tr']
            end_tr = sub_dic_behav[key][mov_name][f'run-{run:d}']['end_tr']
            run_slice = index_img(fmri_run, slice(start_tr + trim_start, end_tr)) # end_tr incoroporates the 4 TR buffer
            print(f'fMRI shape: {run_slice.shape}')
            
            # BEHAV checks #
            assert mov_runs.shape[0] == mov_len_dic[mov_name] + tr_buffer - trim_start, f'behavioral ERROR: movie: {mov_name}, epi {epi_run}, run: {run}'

            # fMRI data check #
            assert run_slice.shape[3] == mov_runs.shape[0] + tr_buffer - trim_start, f'fMRI error! {run_slice.shape[3] + trim_start} verse {mov_runs.shape[0]}'

            
            # Get parcels and create 2d 
            print(f'shape of image {run_slice.shape}')
            f_dat_2d = run_slice.get_fdata()
            
            # loop through all TRs and get mean of each voxel pattern #
            means = [np.mean(f_dat_2d[atlas_img == parcel, :], axis=0)
                            for parcel in np.unique(atlas_img)[1:]]
            print(f'shape of extracted means: {np.column_stack(means).shape}')
            
            # expand to three dimensions for stacking 
            mov_means = np.expand_dims(np.column_stack(means), 2)
            print(f'expand! {mov_means.shape}')

            # Stack the run slice with the mov_runs array
            mov_runs = np.dstack((mov_runs, mov_means))
            print(f'stacked! {mov_runs.shape}')
        
        # set outer loop #
        if key == 'External':
            target_dict = external
        else:
            target_dict = internal

        if mov_name not in target_dict:
            target_dict[mov_name] = mov_runs
        else:
            #mov_runs = np.expand_dims(mov_runs, 3)
            target_dict[mov_name] = np.dstack((target_dict[mov_name], mov_runs))
    print(f'\n subject {sub} finished \n')

            
print('done')
            
            
            
            
            
            

# Define main function

In [23]:
def get_activations_tempnorm(sub_list, roi, roi_voxels, tr_range, cond_list):
    """
    Purpose:
        - extract activations from each ROI at each window, average two TRs within window
    Inputs:
        - sub_list: list of subs to iterate through
        - roi: which roi to extract activations 
        - roi_voxels: number of voxels for the selected ROI
        - tr_range: how many TRs should be averaged over?
        - cond_list: Which two conditions are we comparing
    Outputs:
        - X: Activations for every button press across subs for a given window 
        - Y: array of integers describing if button press is cond a (0) or cond b (1)
        - Groups: array of integers denoting button presses for each sub
    """
    # load in target ROI
    roi = roi
    high_pass = 1/128
    roi_mask = load_roi_mask(roi[0], "MNI")

    # Create masker object and REGRESS CONFOUNDS 
    masker = NiftiMasker(mask_img=roi_mask,smoothing_fwhm=2,
                         standardize=True, detrend=True, high_pass=high_pass,t_r=1.5)

    ## X variable corresponds to a matrix that is: roi voxels x 
    X = np.empty((0, roi_voxels))
    ## Set subject array ## 
    group_id = 0
    groups = np.array([])
    ## set array for labels ## 
    y = np.array([])

    for sub in sub_list:
        ## Which conditions? ## 
        cond_a = cond_list[0]
        cond_b = cond_list[1]

        print(sub)
        # LOAD RUNS FOR COND A AND COND B
        fmri_imgs = load_smsc_fmri_dic(sub,behav_p,cond_a,cond_b,suffix,int_mask)
        # for each condition, extract patterns for each TR and average
        for sing_cond in cond_list:
            # for each run in available run dictionairy
            for run in range(len(run_dic[sub][sing_cond])):
                temp_run = []
                print("condition: ", sing_cond)
                #temp_pred = []
                print("run", run)

                # Fit the masker object to extract a 2d matrix: voxels x TR [454, 205]
                print("sub: ", sub)
                roi_act = masker.fit_transform(fmri_imgs[sing_cond][run], confounds = conf_sub[sub][sing_cond][run])
                print(roi_act.shape)


                # extract BPRESS behavioral data # 
                linez = bpress[sub][sing_cond][run]
                print("button press TRS:", linez)

                # Average activations 
                avg_activations = []
                
                # For each TR #
                for tr in linez:
                    # Find the tr that each onset occured - convert from seconds to TR
                    print(tr)
                    tr = round(tr/1.5)
                    print(tr)
                    temp = []
                    for i in tr_range:
                        try:
                            # IF tr exists in the range of TRs add to temp array before averaging in the next step
                            temp.append(roi_act[tr+i])
                        except:
                            continue
                    print("temp length: ", len(temp))
                    # If TRs exist for this button press, add it to the array # 
                    if (len(temp) > 0):
                        X = np.vstack((X, np.mean(temp, axis=0)))
                        print("X: ", X.shape)
                        groups = np.append(groups, group_id)
                        # note that I changed this to cond_a, cond_b
                        if (sing_cond == cond_a):
                            y = np.append(y, 0)

                        elif (sing_cond == cond_b):
                            y = np.append(y, 1)


        group_id = group_id + 1
    
    return X, y, groups

In [24]:
def get_activations_spacenorm(sub_list, roi, roi_voxels, tr_range, cond_list):
    """
    Purpose:
        - extract activations from each ROI at each window, average two TRs within window
        - ** only difference from above is we changed standardize to false **
    Inputs:
        - sub_list: list of subs to iterate through
        - roi: which roi to extract activations 
        - roi_voxels: number of voxels for the selected ROI
        - tr_range: how many TRs should be averaged over?
        - cond_list: Which two conditions are we comparing
    Outputs:
        - X: Activations for every button press across subs for a given window 
        - Y: array of integers describing if button press is cond a (0) or cond b (1)
        - Groups: array of integers denoting button presses for each sub
    """
    # load in target ROI
    high_pass = 1/128
    roi_mask = load_roi_mask(roi[0], "MNI")

    # Create masker object and REGRESS CONFOUNDS 
    masker = NiftiMasker(mask_img=roi_mask,smoothing_fwhm=2,
                         standardize=False, detrend=True, high_pass=high_pass,t_r=1.5)

    ## X variable corresponds to a matrix that is: roi voxels x 
    X = np.empty((0, roi_voxels))
    ## Set subject array ## 
    group_id = 0
    groups = np.array([])
    ## set array for labels ## 
    y = np.array([])

    for sub in sub_list:
        ## Which conditions? ## 
        cond_a = cond_list[0]
        cond_b = cond_list[1]

        print(sub)
        # LOAD RUNS FOR COND A AND COND B
        fmri_imgs = load_smsc_fmri_dic(sub,behav_p,cond_a,cond_b,suffix,int_mask)
        # for each condition, extract patterns for each TR and average
        for sing_cond in cond_list:
            # for each run in available run dictionairy
            for run in range(len(run_dic[sub][sing_cond])):
                temp_run = []
                print("condition: ", sing_cond)
                #temp_pred = []
                print("run", run)

                # Fit the masker object to extract a 2d matrix: voxels x TR [454, 205]
                print("sub: ", sub)
                roi_act = masker.fit_transform(fmri_imgs[sing_cond][run], confounds = conf_sub[sub][sing_cond][run])
                print(roi_act.shape)


                # extract BPRESS behavioral data # 
                linez = bpress[sub][sing_cond][run]
                print("button press TRS:", linez)

                # Average activations 
                avg_activations = []
                
                # For each TR #
                for tr in linez:
                    # Find the tr that each onset occured - convert from seconds to TR
                    print(tr)
                    tr = round(tr/1.5)
                    print(tr)
                    temp = []
                    for i in tr_range:
                        try:
                            # IF tr exists in the range of TRs add to temp array before averaging in the next step
                            temp.append(roi_act[tr+i])
                        except:
                            continue
                    print("temp length: ", len(temp))
                    # If TRs exist for this button press, add it to the array # 
                    if (len(temp) > 0):
                        X = np.vstack((X, np.mean(temp, axis=0)))
                        print("X: ", X.shape)
                        groups = np.append(groups, group_id)
                        # note that I changed this to cond_a, cond_b
                        if (sing_cond == cond_a):
                            y = np.append(y, 0)

                        elif (sing_cond == cond_b):
                            y = np.append(y, 1)


        group_id = group_id + 1
    
    return X, y, groups

# begin

In [25]:
# Top Directory
top_dir = '/jukebox/graziano/coolCatIsaac/ATM/code/analysis/MVPA/final'
act_dir = top_dir + '/activations'
perm_dir = top_dir +'/permutations'
perm_results = perm_dir + '/perm_results'

In [26]:
### Set RoI ## 
#roi_dict = {'motor': ["l_prim_motor_sm_win2_mask"], 'dmPFC': ["dmPFC_ovlp_mask"], 'rTPJ': ["rTPJ_mask"]}
#roi_dict = {'rTPJ_2' : ['RH_Cont_Par_2'], 'dmPFC': ["dmPFC_ovlp_mask"],'rTPJ': ["rTPJ_mask"]}
roi_dict = { 'dmPFC': ["dmPFC_ovlp_mask"],'rTPJ': ["rTPJ_mask"]}


cond_list = [['SM', 'SC'], ['OM', 'OC']]
voxels_dict = {'motor': 454, 'dmPFC': 112, 'rTPJ': 187, 'rTPJ_2': 198}

## Final 

tr_range_dict = {'1' : np.array([-2,-3]), '2' : np.array([0,-1]), '3' : np.array([1,2]), 
                 '4' : np.array([3,4]), '5' : np.array([5,6]), '6' : np.array([7,8]), 
                '7' : np.array([9,10]), '8' : np.array([11,12])}


### Save Vars ## 
date = '2023-06-06'
act_dict_name = f'roi_activations_final_n28_tempnorm_{date}'

In [29]:
sub_list = ['sub-000', 'sub-004']

In [30]:
#### CHECK BELOW FOR ZSCORE ####

In [31]:
for roi_name in roi_dict:
    roi = roi_dict[roi_name]
    print(roi_name)
    for conds in cond_list:
        print(conds)
        for window in tr_range_dict:
            print(f'window {window}')
            # Get activations ---- norm by space or time ? --- #
            X, y, g = get_activations_tempnorm(sub_list, roi, roi_voxels = voxels_dict[roi_name], 
                                      tr_range = tr_range_dict[window], cond_list = conds)
            ##### NORMALIZE ACROSS VOX ####
            #X = zscore(X, axis = 1)
            ###
            print(X.shape)
            print(y.shape)
            print(g.shape)
            
            print('now save')
            # set analysis # 
            analysis = '%s_%s_%s_win%s' %(conds[0], conds[1], roi_name, window)
            # Save act dic ## 
            if os.path.isfile(os.path.join(act_dir, '%s.npy') %(act_dict_name)):
                print('exists!')
                activations = np.load(os.path.join(act_dir, '%s.npy') %(act_dict_name ), allow_pickle=True).item()
            else:
                activations = {}
                print('creating dic')
            d2 = {"X": X, "y": y, "groups":g}
            activations[analysis] = d2
            #np.save(os.path.join(act_dir, '%s.npy') %(act_dict_name), activations)

            
        

dmPFC
['SM', 'SC']
window 1
expected shape: 78, 93,65
mask shape:  (78, 93, 65)
Loaded dmPFC_ovlp_mask mask
sub-000
concatenated SM SC
condition:  SM
run 0
sub:  sub-000
(205, 112)
button press TRS: [ 94.6  115.6  154.02 221.83 269.04]
94.6
63
temp length:  2
X:  (1, 112)
115.6
77
temp length:  2
X:  (2, 112)
154.02
103
temp length:  2
X:  (3, 112)
221.83
148
temp length:  2
X:  (4, 112)
269.04
179
temp length:  2
X:  (5, 112)
condition:  SM
run 1
sub:  sub-000
(205, 112)
button press TRS: [ 27.81  59.33  74.99 103.14 126.75 154.29 167.91 190.39 208.15 225.51
 251.94 276.34 299.11]
27.81
19
temp length:  2
X:  (6, 112)
59.33
40
temp length:  2
X:  (7, 112)
74.99
50
temp length:  2
X:  (8, 112)
103.14
69
temp length:  2
X:  (9, 112)
126.75
84
temp length:  2
X:  (10, 112)
154.29
103
temp length:  2
X:  (11, 112)
167.91
112
temp length:  2
X:  (12, 112)
190.39
127
temp length:  2
X:  (13, 112)
208.15
139
temp length:  2
X:  (14, 112)
225.51
150
temp length:  2
X:  (15, 112)
251.94
168
te

(205, 112)
button press TRS: [ 18.28  37.75  54.77  71.11  88.27 104.54 121.74 137.73 155.52 172.64
 189.77 206.53 222.51 237.14 254.9  273.37 292.08]
18.28
12
temp length:  2
X:  (59, 112)
37.75
25
temp length:  2
X:  (60, 112)
54.77
37
temp length:  2
X:  (61, 112)
71.11
47
temp length:  2
X:  (62, 112)
88.27
59
temp length:  2
X:  (63, 112)
104.54
70
temp length:  2
X:  (64, 112)
121.74
81
temp length:  2
X:  (65, 112)
137.73
92
temp length:  2
X:  (66, 112)
155.52
104
temp length:  2
X:  (67, 112)
172.64
115
temp length:  2
X:  (68, 112)
189.77
127
temp length:  2
X:  (69, 112)
206.53
138
temp length:  2
X:  (70, 112)
222.51
148
temp length:  2
X:  (71, 112)
237.14
158
temp length:  2
X:  (72, 112)
254.9
170
temp length:  2
X:  (73, 112)
273.37
182
temp length:  2
X:  (74, 112)
292.08
195
temp length:  2
X:  (75, 112)
condition:  SC
run 1
sub:  sub-004
(205, 112)
button press TRS: [ 20.32  36.63  52.94  71.46  88.05 105.7  121.53 141.96 157.6  174.38
 192.   208.07 228.53 241.04 25

KeyboardInterrupt: 

In [32]:
X.shape

(93, 112)

In [54]:
activations = np.load(os.path.join(act_dir, '%s.npy') %(act_dict_name ), allow_pickle=True).item()
for key in sorted(activations):
    print(key)

OM_OC_dmPFC_win1
OM_OC_dmPFC_win2
OM_OC_dmPFC_win3
OM_OC_dmPFC_win4
OM_OC_dmPFC_win5
OM_OC_dmPFC_win6
OM_OC_dmPFC_win7
OM_OC_dmPFC_win8
OM_OC_rTPJ_win1
OM_OC_rTPJ_win2
OM_OC_rTPJ_win3
OM_OC_rTPJ_win4
OM_OC_rTPJ_win5
OM_OC_rTPJ_win6
OM_OC_rTPJ_win7
OM_OC_rTPJ_win8
SM_SC_dmPFC_win1
SM_SC_dmPFC_win2
SM_SC_dmPFC_win3
SM_SC_dmPFC_win4
SM_SC_dmPFC_win5
SM_SC_dmPFC_win6
SM_SC_dmPFC_win7
SM_SC_dmPFC_win8
SM_SC_rTPJ_win1
SM_SC_rTPJ_win2
SM_SC_rTPJ_win3
SM_SC_rTPJ_win4
SM_SC_rTPJ_win5
SM_SC_rTPJ_win6
SM_SC_rTPJ_win7
SM_SC_rTPJ_win8
